************************
Recherche dans une table
************************

In [2]:
#Fichier de travail
import csv

FILE1 = '../data/bac.csv'

f = open(FILE1, newline='', encoding='utf8')
table = [dict(row) for row in csv.DictReader(f, delimiter=';')]

# Cohérence d'une table

## Les données sont-elles valides?
Dans le cours précédent on a importé les données d'un fichier csv dans un tableau. Cependant, rien ne garantit la validité des lignes.  Par exemple, on souhaiterait que les champs 'taux_..' soient complets et que leur valeur représente bien un nombre (avant sa conversion en `int` par la suite). Par ailleurs, le taux ne doit pas être supérieur à 100. D'où un exemple de fonction de validation:

In [3]:
def validation(ligne):
    """
    Retourne un booléen valant True uniquement si l'année et les taux sont des nombres (inférieurs 
    à 100 pour les taux);
    ligne: dictionnaire
    """
    an, tL, tES, tS = ligne['annee'], ligne['taux_L'], ligne['taux_ES'], ligne['taux_S']
    #La méthode de chaine isdecimal() est particulièrement adaptée ici (voir aide)
    sont_des_nb = an.isdecimal() and tL.isdecimal() and tES.isdecimal() and tS.isdecimal()
    #Si les taux sont des nombres, on vérifie leur conformité.
    return sont_des_nb and int(tL) <= 100 and int(tES) <= 100 and int(tS) <= 100

In [4]:
help(str.isdecimal)

Help on method_descriptor:

isdecimal(self, /)
    Return True if the string is a decimal string, False otherwise.
    
    A string is a decimal string if all characters in the string are decimal and
    there is at least one character in the string.



On peut alors construire une table ayant des lignes conformes aux critères présentés ci-dessus et afficher les 5 premières lignes:

In [5]:
table_conforme = [ligne for ligne in table if validation(ligne)]
for i in range(5):
    print(table_conforme[i])

{'id_lycee': 'LYCEE MONTMAJOUR', 'annee': '2012', 'ville': 'ARLES', 'taux_L': '95', 'taux_ES': '79', 'taux_S': '96', 'academie': 'AIX-MARSEILLE'}
{'id_lycee': 'LYCEE BRISTOL', 'annee': '2017', 'ville': 'CANNES', 'taux_L': '84', 'taux_ES': '84', 'taux_S': '75', 'academie': 'NICE'}
{'id_lycee': 'LYCEE MARGUERITE DE NAVARRE', 'annee': '2017', 'ville': 'BOURGES', 'taux_L': '89', 'taux_ES': '94', 'taux_S': '91', 'academie': 'ORLEANS-TOURS'}
{'id_lycee': 'LYCEE FULBERT', 'annee': '2017', 'ville': 'CHARTRES', 'taux_L': '88', 'taux_ES': '88', 'taux_S': '89', 'academie': 'ORLEANS-TOURS'}
{'id_lycee': 'LYCEE NOTRE DAME', 'annee': '2017', 'ville': 'CHARTRES', 'taux_L': '100', 'taux_ES': '97', 'taux_S': '96', 'academie': 'ORLEANS-TOURS'}


## Comment changer le type de certains attributs?
En plus de la validation des données, on souhaiterait convertir certains attributs (l'année et les taux) en `int`.
Une fonction de conversion a déjà été vue dans la fiche d'exercices du cours précédent. En voici une autre:

In [6]:
def conversion(enr):
    """
    Retourne un dictionnaire avec les champs 'Année' et 'Taux_..' convertis en 'int'.
    enr: un enregistrement, de type dictionnaire
    """
    
    assert isinstance(enr, dict), "La donnée convertie doit être un dictionnaire"
    return {'Etablissement':enr['id_lycee'],
           'Année':int(enr['annee']),
           'Ville':enr['ville'],
           'Taux_L':int(enr['taux_L']),
           'Taux_ES':int(enr['taux_ES']),
           'Taux_S':int(enr['taux_S']),
           'Académie':enr['academie']}

In [7]:
table_validee = [conversion(ligne) for ligne in table if validation(ligne)]

In [8]:
for i in range(5):
    print(table_validee[i])

{'Etablissement': 'LYCEE MONTMAJOUR', 'Année': 2012, 'Ville': 'ARLES', 'Taux_L': 95, 'Taux_ES': 79, 'Taux_S': 96, 'Académie': 'AIX-MARSEILLE'}
{'Etablissement': 'LYCEE BRISTOL', 'Année': 2017, 'Ville': 'CANNES', 'Taux_L': 84, 'Taux_ES': 84, 'Taux_S': 75, 'Académie': 'NICE'}
{'Etablissement': 'LYCEE MARGUERITE DE NAVARRE', 'Année': 2017, 'Ville': 'BOURGES', 'Taux_L': 89, 'Taux_ES': 94, 'Taux_S': 91, 'Académie': 'ORLEANS-TOURS'}
{'Etablissement': 'LYCEE FULBERT', 'Année': 2017, 'Ville': 'CHARTRES', 'Taux_L': 88, 'Taux_ES': 88, 'Taux_S': 89, 'Académie': 'ORLEANS-TOURS'}
{'Etablissement': 'LYCEE NOTRE DAME', 'Année': 2017, 'Ville': 'CHARTRES', 'Taux_L': 100, 'Taux_ES': 97, 'Taux_S': 96, 'Académie': 'ORLEANS-TOURS'}


## La table comporte-t-elle des doublons?

Les tables de données ne doivent généralement pas comporter de doublons. La fonction suivante vérifie la présence de doublons dans une table.

In [9]:
def doublon(table):
    """
    Retourne un booléen correspond à la présence ou non de doublons dans la table.
    table: tableau de dictionnaires ou de tableaux
    """
    for i in range(len(table)):
        for j in range(i + 1, len(table)):
            if table[i] == table[j]:
                return True
    return False

In [10]:
doublon(table_validee)

False

*Note*  
Cette fonction n'est pas très efficace, notamment sur des tables volumineuses.

# Sélection de lignes ou de colonnes

## Comment sélectionner des lignes d'une table?
Les opérations faites sur les tables sont en nombre très limité. Parmi celles-ci, on trouve la **sélection** de lignes qui répondent à certain(s) critère(s). Ces critères sont exprimés avec des booléens.  
Afin de faciliter l'écriture et l'évaluation des critères, on utilisera la fonction `eval` de python, dont l'aide est fournie ci-dessous.

In [11]:
help(eval)

Help on built-in function eval in module builtins:

eval(source, globals=None, locals=None, /)
    Evaluate the given source in the context of globals and locals.
    
    The source may be a string representing a Python expression
    or a code object as returned by compile().
    The globals must be a dictionary and locals can be any mapping,
    defaulting to the current globals and locals.
    If only globals is given, locals defaults to it.



In [12]:
def select(table, critere):
    """
    Retourne une table dont les lignes vérifient le critere passé en paramètre.
    table: tableau de dictionnaires
    critere: chaine exprimant la condition 
    """
    return [ligne for ligne in table if eval(critere)]

In [13]:
select(table_validee, "'OLIVE' in ligne['Etablissement'] and ligne['Année'] >= 2017")

[{'Etablissement': "LYCEE DE BOIS D'OLIVE (GENERAL ET TECHNO.)",
  'Année': 2017,
  'Ville': 'ST PIERRE',
  'Taux_L': 96,
  'Taux_ES': 79,
  'Taux_S': 94,
  'Académie': 'LA REUNION'},
 {'Etablissement': "LYCEE DE BOIS D'OLIVE (GENERAL ET TECHNO.)",
  'Année': 2018,
  'Ville': 'ST PIERRE',
  'Taux_L': 83,
  'Taux_ES': 91,
  'Taux_S': 97,
  'Académie': 'LA REUNION'}]

In [14]:
def selection(table, academie, annee):
    critere = '"' + academie + '"' + \
    " in ligne['Académie'] and ligne['Année'] == " + str(annee)
    for resultat in [ligne for ligne in table if eval(critere)]:
        print(resultat)

In [15]:
selection(table_validee, 'LA REUNION', 2012)

{'Etablissement': 'LYCEE SARDA GARRIGA', 'Année': 2012, 'Ville': 'ST ANDRE', 'Taux_L': 100, 'Taux_ES': 94, 'Taux_S': 92, 'Académie': 'LA REUNION'}
{'Etablissement': 'LYCEE LE VERGER', 'Année': 2012, 'Ville': 'STE MARIE', 'Taux_L': 75, 'Taux_ES': 83, 'Taux_S': 83, 'Académie': 'LA REUNION'}
{'Etablissement': 'LYCEE DE VINCENDO (GENERAL ET TECHNO.)', 'Année': 2012, 'Ville': 'ST JOSEPH', 'Taux_L': 91, 'Taux_ES': 86, 'Taux_S': 91, 'Académie': 'LA REUNION'}
{'Etablissement': 'LYCEE CATHOLIQUE LEVAVASSEUR', 'Année': 2012, 'Ville': 'STE CLOTILDE', 'Taux_L': 95, 'Taux_ES': 100, 'Taux_S': 100, 'Académie': 'LA REUNION'}
{'Etablissement': 'LYCEE PIERRE LAGOURGUE (GENERAL ET TECHNO.)', 'Année': 2012, 'Ville': 'LE TAMPON', 'Taux_L': 64, 'Taux_ES': 78, 'Taux_S': 90, 'Académie': 'LA REUNION'}
{'Etablissement': 'LYCEE BELLEPIERRE', 'Année': 2012, 'Ville': 'ST DENIS CEDEX', 'Taux_L': 72, 'Taux_ES': 88, 'Taux_S': 96, 'Académie': 'LA REUNION'}
{'Etablissement': 'LYCEE GEORGES BRASSENS', 'Année': 2012, 'Vi

## Comment sélectionner des colonnes d'une table?

La sélection de colonnes, appelé couramment **projection** peut être réalisée avec le code de la cellule suivante. Supposons que l'on veuille garder uniquement les colonnes 'Ville' et 'Académie' de notre table. Cette opération peut être facilement réalisée avec un tableau construit en compréhension.

In [23]:
t = [{'Ville':ligne['Ville'], 'Académie':ligne['Académie']} for ligne in table_validee]
#Affichage des 5 premières lignes
for i in range(5):
    print(t[i])

{'Ville': 'ARLES', 'Académie': 'AIX-MARSEILLE'}
{'Ville': 'CANNES', 'Académie': 'NICE'}
{'Ville': 'BOURGES', 'Académie': 'ORLEANS-TOURS'}
{'Ville': 'CHARTRES', 'Académie': 'ORLEANS-TOURS'}
{'Ville': 'CHARTRES', 'Académie': 'ORLEANS-TOURS'}


### Bonus: une fonction projection

In [24]:
def projection(table, tableau_colonnes):
    """
    Retourne une table ne contenant uniquement que les colonnes présentes dans tableau_colonnes.
    table: tableau de dictionnaires
    tableau_colonnes: tableau de chaine de caractères correspondant aux noms des colonnes sélectionnées.
    """
    return [{nom_col:ligne[nom_col] for nom_col in tableau_colonnes} for ligne in table]

#### Explication

La fonction `projection` retourne un tableau de dictionnaires construit en **compréhension**. Décomposons cette construction.  

Entre les crochets `[...]`, on distingue bien trois parties essentielles: `{nom_col:ligne[nom_col] for nom_col in tableau_colonnes}`, `for` et `ligne in table`. Ce bloc d'instructions est du type `expression(ligne) for ligne in table`. Il est responsable de la formation des différentes lignes `{...}, {...}, ..., {...}`.     

La suite d'instructions `{nom_col:ligne[nom_col] for nom_col in tableau_colonnes}` est responsable de la construction du **contenu** de chaque ligne (*il s'agit de la construction d'un dictionnaire par compréhension*). Les clés de chaque dictionnaire formé sont situées dans le tableau passé en argument et les valeurs sont les champs associés, de la ligne de la table de données.

#### Exemples d'utilisation

In [25]:
#Affichage des 5 premiers éléments d'une table contenant ville et académie

t = projection(table_validee, ['Ville', 'Académie', 'Etablissement'])
for i in range(5):
    print(t[i])

{'Ville': 'ARLES', 'Académie': 'AIX-MARSEILLE', 'Etablissement': 'LYCEE MONTMAJOUR'}
{'Ville': 'CANNES', 'Académie': 'NICE', 'Etablissement': 'LYCEE BRISTOL'}
{'Ville': 'BOURGES', 'Académie': 'ORLEANS-TOURS', 'Etablissement': 'LYCEE MARGUERITE DE NAVARRE'}
{'Ville': 'CHARTRES', 'Académie': 'ORLEANS-TOURS', 'Etablissement': 'LYCEE FULBERT'}
{'Ville': 'CHARTRES', 'Académie': 'ORLEANS-TOURS', 'Etablissement': 'LYCEE NOTRE DAME'}


In [26]:
#Recherche de doublons
doublon(projection(table_validee, ['Ville', 'Académie', 'Etablissement']))

True

**Attention: une projection peut entrainer la présence de doublons**

---
B. DARID
![licence](img/Cc-by-nc_icon.svg.png)